# Flownets
<p>Flownets are a method of determining groundwater flow that follows a nonuniform path, such as flow down and under a dam. Unlike in the first formulations of groundwater flow, vertical flow is incorporated into this model. The purpose of a flownet is to display equipotential lines on a graph and separate the flow into separate stream tubes.</p>
<br>
<p>For this example, I use a vertical cross-section with a dam in the center. The hydraulic head is higher on one side than the other, and flow is directed down and under the dam. Just like before, groundwater flows from regions of high hydraulic head to low, though now it may also flow downward. This means that the hydraulic head in a single column is not the same, which causes water to flow downwards on the region with higher head and downwards in the region with lower head.</p>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np